In [ ]:
import pandas as pd

# Motivation

We have chosen a dataset with country wide traffic accidents in the US. This dataset has a lot of data that we can explore, while using the multitude of the tools we have taught in the course. It also opens up the possibility to do advanced analysis. Our goal is to show the user how dangerous it can be to drive during bad/extreme weather conditions, using interactive tools. We believe that by doing this, the user is more entertained and invested in the findings of our analysis.

# Basic stats

Before starting the analysis, we have had to remove some of the non-important features of our dataset. We did this by removing 26 attributes. Furthermore some cleanup was needed to remove n/a values. 

MANGLER: "Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis."

In [ ]:
# data = pd.read_csv("data/US_Accidents_Dec20.csv")
# CleanedData = data.drop(['Number','Distance(mi)','Airport_Code','Street','Side','Country','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight', 'End_Lat','End_Lng'],axis='columns', inplace=False)
# data.info()
# CleanedDataV2 = CleanedData.dropna()

data = pd.read_csv("data/CleanedData.csv")
mean_severity = df["weight"].mean()
mean_temperature = df["Temperature(F)"].mean()
mean_severity = df["weight"].mean()
mean_severity = df["weight"].mean()
mean_severity = df["weight"].mean()


# Data Analysis



# Genre

We will create a userbased interface, which will enables the user to investigate the data by ones self. But also to help the user understand the data we will display the data in a "cartoon" type, telling our story of the analysis, in order to inspire the user, for how the data can be investigated.


# Visualizations

We have chosen to use a mix of different plots, where our most important visualizations are when using geodata. This type of plot is easy to understand and therefore a good choice, because we try to cater to all type of users.


# Discussion



# Contributions



